In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

model_name_or_path = "facebook/opt-6.7b"

quantization_config = GPTQConfig(
     bits=4, # 量化精度
     group_size=128,
     dataset="wikitext2",
     desc_act=False,
)

In [2]:
quant_model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    quantization_config=quantization_config,
    device_map='auto')

Request b33b9586-8249-4463-8e72-f132bf9f8206: HEAD https://hf-mirror.com/facebook/opt-6.7b/resolve/main/config.json (authenticated: False)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Request 46126c79-7cff-4f39-b421-59bb3cfb8231: HEAD https://hf-mirror.com/facebook/opt-6.7b/resolve/main/generation_config.json (authenticated: False)
Request a1c97ea5-8f2e-4b09-8efe-b179ca479551: HEAD https://hf-mirror.com/facebook/opt-6.7b/resolve/main/tokenizer_config.json (authenticated: False)
Request fc729c6c-85fd-456e-9175-f2305f190d00: GET https://hf-mirror.com/api/datasets/wikitext (authenticated: False)
Request 7a785663-2ce1-45ae-81c1-80b06628d1c7: GET https://hf-mirror.com/api/datasets/wikitext (authenticated: False)
Request 9cc31c33-685b-43e7-866a-5ddae6303939: GET https://hf-mirror.com/api/datasets/wikitext/revision/b08601e04326c79dfdd32d625aee71d232d685c3 (authenticated: False)
Request fb416b4c-563c-4096-90bb-d64026510b69: POST https://hf-mirror.com/api/datasets/wikitext/paths-info/b08601e04326c79dfdd32d625aee71d232d685c3 (authenticated: False)
Request 48b9719b-61bc-41cf-81ba-baa1d07da79d: POST https://hf-mirror.com/api/datasets/wikitext/paths-info/b08601e04326c79dfdd32d62

Quantizing model.decoder.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
quant_model.model.decoder.layers[0].self_attn.q_proj.__dict__

In [ ]:
import os
from datetime import datetime

# 获取当前日期和时间
current_dateTime = datetime.now()

# 格式化为 yyyyMMddHHmmss
formatted_dateTime = current_dateTime.strftime("%Y%m%d%H%M%S")
model_save_path=os.environ["MODEL_SAVE_PATH"]
save_model_dir = f"{model_save_path}models/{model_name_or_path}/{formatted_dateTime}"
# 保存模型权重
quant_model.save_pretrained(save_model_dir)
print(f"模型已保存:{save_model_dir}")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

In [ ]:
from transformers import AutoModelForCausalLM, GPTQConfig, AutoTokenizer

model_name_or_path = "facebook/opt-2.7b"
custom_dataset = ["auto-gptq is an easy-to-use model quantization library with user-friendly apis, based on GPTQ algorithm."]

custom_quantization_config = GPTQConfig(
    bits=4,
    group_size=128,
    desc_act=False,
    dataset=custom_dataset
)

custom_quant_model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                                          quantization_config=custom_quantization_config,
                                                          torch_dtype=torch.float16,
                                                          device_map="auto")

In [ ]:
text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = custom_quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))